This encompasses the approach to text processing with a more OCR approach.
1. Gray-scale ->  Blur image -> draw ROI of blurred dark area -> Retrieve ROI as Rect -> Extract text using Rect with PyMuPDF
2. Possible drawback:
- Inability to detect text in 

In [30]:
import pymupdf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

import re
import os
import json
from collections import defaultdict
import difflib
import threading

In [31]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from typing import List
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
import heapq
class MedianFinder(object):
    def __init__(self):
        self.minheap = []
        self.maxheap = []

    def addNum(self, num:float):
        if not self.minheap and not self.maxheap:
            heapq.heappush(self.maxheap, -num)
        else:
            if len(self.minheap)== len(self.maxheap):
                if -self.maxheap[0]<=num:
                    heapq.heappush(self.minheap, num)
                    heapq.heappush(self.maxheap, -heapq.heappop(self.minheap))
                else:
                    heapq.heappush(self.maxheap, -num)
            else:
                if -self.maxheap[0]<= num:
                    heapq.heappush(self.minheap, num)
                else:
                    heapq.heappush(self.minheap, -heapq.heappop(self.maxheap))
                    heapq.heappush(self.maxheap, -num)
                
    def findMedian(self):
        if len(self.minheap)==len(self.maxheap):
            return (self.minheap[0]-self.maxheap[0])/2.0
        else:
            return -self.maxheap[0]

In [33]:

class Node:
    def __init__(self, content):
        self.content = content
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def __str__(self):
        return f"Node({self.content}, {self.children})"

In [34]:
from typing import Tuple


def should_merge(rect1, rect2, h_threshold=4, v_threshold=1) -> Tuple[str, bool]:
    """
    Determine if two rectangles should be merged based on proximity and alignment.
    
    Args:
        rect1, rect2: PyMuPDF Rect objects
        h_threshold: Maximum horizontal gap between regions to consider merging
        v_threshold: Maximum vertical gap between regions to consider merging
        line_height_factor: Factor to determine if rects are on same line
        
    Returns:
        str: Type of merge ('horizontal', 'vertical', 'none')
        bool: True if rectangles should be merged
    """
    # Check if rectangles are on approximately the same line
    height1 = rect1.height
    height2 = rect2.height

    avg_height = (height1 + height2) / 2
    
    same_line = abs(rect1.y0 - rect2.y0) < avg_height * 0.75 
    # Check horizontal proximity
    if rect1.x1 < rect2.x0: 
        h_gap = rect2.x0 - rect1.x1
        return same_line and h_gap < h_threshold
    elif rect2.x1 < rect1.x0:  
        h_gap = rect1.x0 - rect2.x1
        return same_line and h_gap < h_threshold
    
    # Merging Intervals (LC)
    # Check if they completely overlap horizontally
    overlap_x = (rect1.x0 > rect2.x0 and rect2.x1 > rect1.x1) \
                or (rect2.x0 > rect1.x0 and rect1.x1 > rect2.x1)
    # if it overlaps horizontally by over 75% of the smaller rect, then consider it overlapping
    overlap_x = overlap_x or (max(rect1.x0, rect2.x0) < min(rect1.x1, rect2.x1) \
                              and max(rect1.x0,rect2.x0) - min(rect1.x1,rect2.x1) > min(rect1.width,rect2.width)*0.75)
    if overlap_x:
        if rect1.y1 < rect2.y0:
            v_gap = rect2.y0 - rect1.y1
            return v_gap < v_threshold
        elif rect2.y1 < rect1.y0:  
            v_gap = rect1.y0 - rect2.y1
            return v_gap < v_threshold 
    
    return overlap_x and (max(rect1.y0, rect2.y0) < min(rect1.y1, rect2.y1))

def merge_text_regions(regions, iterations=3):
    """
    Merge text regions that are close to each other.
    
    Args:
        regions: List of PyMuPDF Rect objects
        iterations: Number of merging passes to perform
    Returns:
        list: Merged PyMuPDF Rect objects
    """
    if not regions:
        return []
    
    # Perform multiple iterations of merging to handle chains of regions
    for _ in range(iterations):
        merged = False
        i = 0
        while i < len(regions):
            j = i + 1
            while j < len(regions):
                if should_merge(regions[i], regions[j], h_threshold=6, v_threshold=0.5): # Make this more relative
                    # Merge rectangles
                    merged_rect = pymupdf.Rect(
                        min(regions[i].x0, regions[j].x0),
                        min(regions[i].y0, regions[j].y0),
                        max(regions[i].x1, regions[j].x1),
                        max(regions[i].y1, regions[j].y1)
                    )
                    regions[i] = merged_rect
                    regions.pop(j)
                    merged = True
                else:
                    j += 1
            i += 1
            
        if not merged:
            break
            
    return regions

In [35]:
def extract_text_with_ocr_approach(pdf_path, page_num=0, visual_proof = False):
    """
    Extract text from a PDF using image processing to identify text regions.
    With text region merging to combine related text blocks.
    
    Args:
        pdf_path (str): Path to the PDF file
        page_num (int): Page number to process (0-indexed)
        
    Returns:
        list: Extracted text segments from identified regions
    """
    # Open the PDF
    doc = pymupdf.open(pdf_path)
    page = doc[page_num]
    zoom_factor = 2
    # Convert page to image
    pix = page.get_pixmap(matrix=pymupdf.Matrix(zoom_factor, zoom_factor))
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    
    # Convert to grayscale
    if img.shape[2] >= 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img[:, :, 0]
    # Apply Gaussian blur
    adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 13, 2)
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    
    dilated = cv2.dilate(adaptive, rectKernel, iterations=1)
    
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, rectKernel)
    
    edged = cv2.Canny(closing, 50, 200, apertureSize=3)

    contours, _  = cv2.findContours(
        edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    # Create ROIs from contours
    initial_regions = []
    for cnt in contours:
        
        x, y, w, h = cv2.boundingRect(cnt)
        # Convert back to PDF coordinates (accounting for the zoom factor)
        pdf_x, pdf_y = x/zoom_factor, y/zoom_factor
        pdf_w, pdf_h = w/zoom_factor, h/zoom_factor
        
        # Create PyMuPDF rect
        rect = pymupdf.Rect(pdf_x, pdf_y, pdf_x + pdf_w, pdf_y + pdf_h)
        initial_regions.append(rect)
    
    # Merge text regions that are close to each other
    merged_regions = merge_text_regions(initial_regions, iterations=5)
    
    # Extract text from merged regions
    text_regions = []
    text_height_collection = MedianFinder()
    
    words = page.get_text("words")

    for rect in sorted(merged_regions, key=lambda r: [(r.y1- r.y0) * (r.x1 -r.x0), r.y0, r.x0]):
        text = ""
        idx = 0
        min_text_height = 1000000
        # Add relative padding to the rect to ensure we catch words that are nearby
        padding = 2  # pixels in PDF coordinate space
        rect = pymupdf.Rect(
            rect.x0 - padding,
            rect.y0 - padding,
            rect.x1 + padding,
            rect.y1 + padding
        )
        # Iterate through words and pick out words that fall within the rect
        prev_y = words[0][1] if words else 0
        while words and idx < len(words):
            current_word = words[idx]
            word_rect = pymupdf.Rect(current_word[:4])
            
            if word_rect.intersects(rect) or rect.contains(word_rect):        
                text_height_collection.addNum(word_rect.height)
                min_text_height = min(min_text_height, word_rect.height) 
                text += current_word[4] + (" " if abs(prev_y - current_word[1]) <= abs(current_word[1] - current_word[3])*0.5 else "\n")
                prev_y = current_word[1]
                words.pop(idx)
            else:
                idx += 1
        
        if text.strip():
            if text.strip().isdigit():
                continue
            text_regions.append({"occupy_space": rect,
                                 "content": text, 
                                 "text_height_median": text_height_collection.findMedian(), 
                                 "min_height": min_text_height, 
                                 "page_num": page_num})
    # print(words)
    if visual_proof:
        viz_img = img.copy()
        for item in text_regions:
            rect = item["occupy_space"]
            # Convert back to image coordinates
            x0, y0, x1, y1 = rect.x0*zoom_factor, rect.y0*zoom_factor, rect.x1*zoom_factor, rect.y1*zoom_factor
            cv2.rectangle(viz_img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
        
        plt.figure(figsize=(15, 15))
        plt.subplot(1, 2, 1)
        plt.imshow(edged, cmap='gray')
        plt.title("Processed Image")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(viz_img)
        plt.title(f"Merged Text Regions: {len(text_regions)} blocks")
        plt.axis('off')
        plt.show()
    
    return text_regions[::-1]


In [36]:
%%time
# Example usage
input_pdf_path = r"D:\DATA300\AudioBookSum\pdf\Clopath.pdf"
doc = pymupdf.open(input_pdf_path)
text = ""
median_collection = []
content = []
i= 1
possible_water_mark = None
has_wm = False
for i in range(len(doc[:5])):
    extracted_pages = sorted(extract_text_with_ocr_approach(input_pdf_path, page_num =i, visual_proof=False), key = lambda x: [x["occupy_space"].y0, x["occupy_space"].x0])
    # save all reapeating elements
    if possible_water_mark and extracted_pages[-1]["content"] == possible_water_mark:
        has_wm = True
    if not extracted_pages:
        continue


    possible_water_mark = extracted_pages[-1]["content"]


for i in range(len(doc)):
    print(f"Processing page {i+1}/{len(doc)}...")
    extracted_pages = sorted(extract_text_with_ocr_approach(input_pdf_path, page_num =i, visual_proof=False), key = lambda x: [x["occupy_space"].y0, x["occupy_space"].x0])
    if has_wm:
        extracted_pages.pop(0)
        extracted_pages.pop()
    
    content.extend(extracted_pages)

# Classify the text regions based on their height and position
content.sort(key =lambda x: ( x["page_num"], x["occupy_space"].y0, x["occupy_space"].x0))
# Rank the text regions based on their height and position
base = ""
curr_rank = 1
count_till_match_size = [0] * len(content)
stack = []
for idx, item in enumerate(content):
    while stack and item["text_height_median"] >= content[stack[-1]]["text_height_median"]: # padding:
        i = stack.pop()
        count_till_match_size[i] = idx - i - 1
    stack.append(idx)
while stack:
    i = stack.pop()
    count_till_match_size[i] = len(content) - i - 1

node_list = []
# Graph DFS traversal
for i in content:
    curr = Node(i)
    node_list.append(curr)
    # print(str(curr))


for i in range(len(node_list)):
    if count_till_match_size[i] > 0:
        for j in range(count_till_match_size[i]):
            node_list[i].add_child(node_list[(i + 1) + j])
    visited = set()

chapters = [i for i in node_list if len(i.children) > 0]
print(count_till_match_size)
if not chapters or sum(count_till_match_size) == 0:
    chapters = node_list
    
for chapter in chapters:
    if chapter not in visited:
        # text += str(chapter.content["text_height_median"]) +"," 
        text +=chapter.content["content"] + "\n"
        visited.add(chapter)
        if chapter.content["content"].strip() in ["Notes", "Notes and References", "Note"]:

            continue
        for child in chapter.children:
            if child not in visited:
                # text += "\t" + str(child.content["min_height"]) +"," 
                text += "\t" + child.content["content"] + "\n"
                visited.add(child)
# print(text)
        


Processing page 1/4...
Processing page 2/4...
Processing page 3/4...
Processing page 4/4...
[0, 0, 11, 0, 0, 0, 7, 0, 2, 1, 0, 2, 1, 0]
CPU times: total: 984 ms
Wall time: 1.37 s


In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text_blocks, chunk_size=1000, chunk_overlap=100):
    """
    Chunk text blocks into smaller pieces while preserving page and bounding box information.
    
    Args:
        text_blocks (list): List of dictionaries containing text with metadata (page, bbox, etc.)
        chunk_size (int): Maximum size of each chunk in characters
        chunk_overlap (int): Number of overlapping characters between chunks
        
    Returns:
        list: List of dictionaries containing chunks with preserved metadata
    """
    chunked_data = []  # Initialize as a list, not a dictionary
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    # First combine all text blocks into a single document string for chunking
    combined_text = ""
    metadata_map = {}  # Map character positions to original metadata
    current_position = 0
    
    # Process each text block and map character positions to metadata
    for idx, block in enumerate(text_blocks):
        # Get text and metadata
        text = block.get("content", "")
        if not text:
            continue
            
        block_length = len(text)
        page = block.get("page_num", 0)
        bbox = block.get("occupy_space", None)
        text_height = block.get("text_height_median", None)
        
        # Map each character position in the combined text to its source metadata
        for i in range(current_position, current_position + block_length):
            metadata_map[i] = {
                "page": page,
                "bbox": bbox,
                "text_height": text_height,
                "original_block_index": idx  # Use idx instead of text_blocks.index()
            }
        
        # Add text to combined string with a marker between blocks
        combined_text += text + " [BLOCK_BREAK] "
        current_position += block_length + len(" [BLOCK_BREAK] ")
    
    # Split the combined text into chunks
    chunks = text_splitter.split_text(combined_text)
    
    # For each chunk, determine the predominant page and bounding box
    chunk_start = 0
    for i, chunk in enumerate(chunks):
        # Find the start position of this chunk in the combined text
        if i == 0:
            start_pos = 0
        else:
            # Look for chunk in the vicinity of where we expect it
            search_start = max(0, chunk_start - chunk_overlap)
            search_text = combined_text[search_start:search_start + len(chunk) + 100]
            rel_pos = search_text.find(chunk[:50])  # Use prefix to locate chunk
            if rel_pos != -1:
                start_pos = search_start + rel_pos
            else:
                # Fallback if exact match not found
                start_pos = chunk_start
        
        # Update for next iteration
        chunk_start = start_pos + len(chunk) - chunk_overlap
            
        # Find the most common page and bounding box in this chunk
        page_counts = {}
        bbox_by_page = {}
        
        for j in range(start_pos, min(start_pos + len(chunk), len(combined_text))):
            if j in metadata_map:
                page = metadata_map[j]["page"]
                if page not in page_counts:
                    page_counts[page] = 0
                    bbox_by_page[page] = []
                    
                page_counts[page] += 1
                if metadata_map[j]["bbox"] and metadata_map[j]["bbox"] not in bbox_by_page[page]:
                    bbox_by_page[page].append(metadata_map[j]["bbox"])
        
        # Find predominant page
        predominant_page = max(page_counts.items(), key=lambda x: x[1])[0] if page_counts else 0
        
        # Create enhanced chunk with metadata
        enhanced_chunk = {
            "text": chunk.replace(" [BLOCK_BREAK] ", "\n\n"),
            "page": predominant_page,
            "bbox_list": bbox_by_page.get(predominant_page, []),
            "start_position": start_pos,
            "chunk_index": i
        }
        
        chunked_data.append(enhanced_chunk)
    
    return chunked_data

In [38]:
def identify_interesting_points(chunked_data, llm, text_blocks, file_name, threshold=5):
    """
    Identify important points in text using LLM, adapted to work with chunk_text output.
    
    Args:
        chunked_data (list): List of dictionaries returned by chunk_text function
        llm: LLM model for generating interesting points
        text_blocks (list): Original text blocks with metadata
        file_name (str): Path to cache file
        threshold (int): Minimum word count for segments to be considered
        
    Returns:
        list: List of interesting sections with metadata and page annotations
    """
    interesting_sections = []
    
    # Return early if chunked_data is empty
    if not chunked_data:
        print("No content identified in the document")
        return interesting_sections
    
    # Create block index for faster lookup - map page numbers to blocks on that page
    page_to_blocks = defaultdict(list)
    for block in text_blocks:
        # Pre-normalize block text once
        content = block.get("content", "")
        if content:
            block["norm_text"] = ' '.join(content.split())
            page_to_blocks[block.get("page_num", 0)].append(block)
    
    # Function to find matching block for a segment with improved fuzzy matching
    def find_matching_block(segment, page_hint=None):
        print(f"Finding matching block for: {segment[:50]} ...") 
        print(f"Page hint: {page_hint}")
        norm_segment = ' '.join(segment.split())
        if not norm_segment:
            return None, False
            
        # First try exact page if provided
        if page_hint is not None:
            for block in page_to_blocks.get(page_hint, []):
                if norm_segment in block["norm_text"]:
                    return block, True
        
        # Try exact matching across all pages
        for page, page_blocks in page_to_blocks.items():
            for block in page_blocks:
                if norm_segment in block["norm_text"]:
                    return block, True
        
        # Try finding any substantial overlap
        best_match = None
        highest_similarity = 0.6  # Lowered threshold for better recall
        best_overlap_len = 0
        
        # First prioritize matches by page if available
        if page_hint is not None:
            for block in page_to_blocks.get(page_hint, []):
                # Try SequenceMatcher similarity
                similarity = difflib.SequenceMatcher(None, norm_segment, block["norm_text"]).ratio()
                if similarity > highest_similarity:
                    highest_similarity = similarity
                    best_match = block
                
                # Try finding common substring
                common_words = set(norm_segment.split()) & set(block["norm_text"].split())
                if len(common_words) > best_overlap_len:
                    best_overlap_len = len(common_words)
                    if not best_match:  # Only replace if we don't have a high similarity match
                        best_match = block
        
        # If no good match on the hinted page, try all pages
        if not best_match or highest_similarity < 0.7:
            for page, page_blocks in page_to_blocks.items():
                for block in page_blocks:
                    similarity = difflib.SequenceMatcher(None, norm_segment, block["norm_text"]).ratio()
                    if similarity > highest_similarity:
                        highest_similarity = similarity
                        best_match = block
                    
                    # Try finding common substring as backup approach
                    common_words = set(norm_segment.split()) & set(block["norm_text"].split())
                    overlap_ratio = len(common_words) / max(1, len(norm_segment.split()))
                    if overlap_ratio > 0.5 and len(common_words) > best_overlap_len:
                        best_overlap_len = len(common_words)
                        if highest_similarity < 0.7:  # Only use word overlap if similarity isn't high
                            best_match = block
        
        # If still no match, but we have a page hint, return any block from that page
        if not best_match and page_hint is not None and page_hint in page_to_blocks and page_to_blocks[page_hint]:
            return page_to_blocks[page_hint][0], False
            
        # If still no match, return any block from any page as last resort
        if not best_match and page_to_blocks:
            first_page = min(page_to_blocks.keys())
            if page_to_blocks[first_page]:
                return page_to_blocks[first_page][0], False
        
        return best_match, False
    
    # Function to extract page number from text with [Page X] annotation
    def extract_page_number(text):
        page_match = re.search(r'\[Page (\d+)\]', text)
        if page_match:
            return int(page_match.group(1)) - 1  # Convert to 0-indexed
        return None
    
    # Cache handling using structured JSON format
    if os.path.exists(file_name):
        try:
            print(f"Loading cached interesting points from {file_name}")
            # Try JSON format first (for files created by the optimized version)
            try:
                with open(file_name, "r", encoding="utf-8") as f:
                    cached_data = json.load(f)
                    
                for segment_data in cached_data.get("segments", []):
                    segment = segment_data.get("text", "")
                    page_hint = segment_data.get("page")
                    
                    if len(segment.split()) < threshold:
                        continue
                    
                    # Check for page number in the text itself
                    text_page = extract_page_number(segment)
                    if text_page is not None:
                        page_hint = text_page
                        # Remove the page annotation if it exists
                        segment_for_matching = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                    else:
                        segment_for_matching = segment
                    
                    matched_block, exact_match = find_matching_block(segment_for_matching, page_hint)
                    if matched_block:
                        # Add page annotation to the text
                        page_num = matched_block.get("page_num", 0)
                        page_annotation = f"[Page {page_num+1}]"
                        
                        # Keep original text but ensure page annotation is correct
                        if text_page is None:
                            clean_segment = segment_for_matching
                            display_text = f"{page_annotation} {clean_segment}"
                        elif text_page != page_num:
                            # Replace with correct page
                            clean_segment = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                            display_text = f"{page_annotation} {clean_segment}"
                        else:
                            display_text = segment  # Keep original if page is correct
                        
                        interesting_sections.append({
                            "page": page_num,
                            "text": display_text,
                            "category": "main",
                            "bbox": matched_block.get("occupy_space"),
                            "page_text": f"Page {page_num+1}"
                        })
                    else:
                        # Still keep the segment but with a default page
                        default_page = page_hint if page_hint is not None else 0
                        page_annotation = f"[Page {default_page+1}]"
                        
                        # Handle page annotation in text
                        if text_page is None:
                            display_text = f"{page_annotation} {segment}"
                        else:
                            # Keep original annotation
                            display_text = segment
                        
                        print(f"Using default page for unmatched segment: {segment[:50]}...")
                        interesting_sections.append({
                            "page": default_page,
                            "text": display_text,
                            "category": "main",
                            "bbox": None,
                            "page_text": f"Page {default_page+1}"
                        })
                
            except json.JSONDecodeError:
                # Fall back to the legacy format for backward compatibility
                with open(file_name, "r", encoding="utf-8") as f:
                    content = f.read()
                
                cached_segments = re.findall(r'```segment(?:\s*-\s*(\d+))?\s*(.*?)```', content, re.DOTALL)
                
                for page_num_str, segment in cached_segments:
                    segment = segment.strip()
                    if len(segment.split()) < threshold:
                        continue
                    
                    # Try to get page number from the text first
                    text_page = extract_page_number(segment)
                    if text_page is not None:
                        page_hint = text_page
                        # Remove the page annotation if it exists
                        segment_for_matching = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                    else:
                        page_hint = int(page_num_str) - 1 if page_num_str else None
                        segment_for_matching = segment
                    
                    matched_block, exact_match = find_matching_block(segment_for_matching, page_hint)
                    
                    if matched_block:
                        # Add page annotation to the text
                        page_num = matched_block.get("page_num", 0)
                        page_annotation = f"[Page {page_num+1}]"
                        
                        # Keep original text but ensure page annotation is correct
                        if text_page is None:
                            clean_segment = segment_for_matching
                            display_text = f"{page_annotation} {clean_segment}"
                        elif text_page != page_num:
                            # Replace with correct page
                            clean_segment = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                            display_text = f"{page_annotation} {clean_segment}"
                        else:
                            display_text = segment  # Keep original if page is correct
                        
                        interesting_sections.append({
                            "page": page_num,
                            "text": display_text,
                            "category": "main",
                            "bbox": matched_block.get("occupy_space"),
                            "page_text": f"Page {page_num+1}"
                        })
                    else:
                        # Still keep the segment but with a default page
                        default_page = page_hint if page_hint is not None else 0
                        page_annotation = f"[Page {default_page+1}]"
                        
                        if text_page is None:
                            display_text = f"{page_annotation} {segment}"
                        else:
                            # Keep original annotation
                            display_text = segment
                        
                        print(f"Using default page for unmatched segment: {segment[:50]}...")
                        interesting_sections.append({
                            "page": default_page,
                            "text": display_text,
                            "category": "main",
                            "bbox": None,
                            "page_text": f"Page {default_page+1}"
                        })
        
        except Exception as e:
            print(f"Error loading cached data: {e}. Generating new segments.")
            interesting_sections = []  # Reset and try generating new
    
    # Generate new segments if no cached results were loaded
    if not interesting_sections:
        try:
            print("Generating new interesting points with LLM...")
            
            prompt_template = PromptTemplate(
                input_variables=["text"],
                template="""
                You are a reasoning summarizer.
                Summarize the provided text and support your summary using different verbatim snippets from the original text.
                Remember:
                The reasoning section must ONLY contain verbatim text from the document.
                Every sentence in the reasoning must be supporting sentences in the summary section.
                Do not add any information that isn't directly from the document.
                IMPORTANT: Do not remove page annotations like [Page X] from your snippets.
                
                Format each important snippet as:
                ```segment - <page number>
                <exact text from the document including the [Page X] annotation>
                ```
                
                Below is text from the main content of a document in English:
                {text}
                """
            )
            
            # Concatenate text from all chunks
            all_text = ""
            for chunk in chunked_data:
                # Add page annotation to make LLM include page numbers
                page_num = chunk["page"]
                all_text += f"[Page {page_num+1}] {chunk['text']}\n\n"
                
            prompt = prompt_template.format(text=all_text)
            
            response = llm.invoke(prompt)
            content = response.content
            print("LLM response received")
            # Extract segments
            segments = re.findall(r'```segment(?:\s*-\s*(\d+))?\s*(.*?)```', content, re.DOTALL)
            print(f"Extracted {len(segments)} segments from LLM response")
            
            # Save in structured JSON format for better caching
            cached_data = {"segments": []}
            
            for page_num_str, segment_text in segments:
                segment = segment_text.strip()
                if len(segment.split()) < threshold:
                    continue
                
                # Try to extract page number from the text content first
                text_page = extract_page_number(segment)
                if text_page is not None:
                    page_hint = text_page
                    print(f"Found page annotation in text: Page {page_hint+1}")
                    # Remove page annotation for matching but keep original text
                    segment_for_matching = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                else:
                    page_hint = int(page_num_str) - 1 if page_num_str else None
                    print(f"Using segment marker page: Page {page_hint+1 if page_hint is not None else 'unknown'}")
                    segment_for_matching = segment
                
                matched_block, exact_match = find_matching_block(segment_for_matching, page_hint)
                
                # Always save the segment, even if no match is found
                if matched_block:
                    page_num = matched_block.get("page_num", 0)
                    page_annotation = f"[Page {page_num+1}]"
                    
                    # Store original segment in cache
                    segment_data = {
                        "page": page_num,
                        "text": segment,
                    }
                    cached_data["segments"].append(segment_data)
                    
                    # For display, ensure we have the correct page number
                    if text_page is None:
                        # No existing page annotation, add one
                        display_text = f"{page_annotation} {segment_for_matching}"
                    elif text_page != page_num:
                        # Replace incorrect page annotation
                        clean_segment = re.sub(r'\[Page \d+\]\s*', '', segment).strip()
                        display_text = f"{page_annotation} {clean_segment}"
                    else:
                        display_text = segment  # Keep original if it had the correct page
                    
                    interesting_sections.append({
                        "page": page_num,
                        "text": display_text,
                        "category": "main",
                        "bbox": matched_block.get("occupy_space"),
                        "page_text": f"Page {page_num+1}"
                    })
                else:
                    # Still save the segment with the hinted page or default to page 0
                    default_page = page_hint if page_hint is not None else 0
                    
                    # Store the original segment in cache
                    segment_data = {
                        "page": default_page,
                        "text": segment,
                    }
                    cached_data["segments"].append(segment_data)
                    
                    # For display, use the page annotation from the text or add a default one
                    if text_page is None:
                        page_annotation = f"[Page {default_page+1}]"
                        display_text = f"{page_annotation} {segment}"
                    else:
                        display_text = segment  # Keep original with its annotation
                    
                    print(f"Using default page {default_page+1} for unmatched segment: {segment[:50]}...")
                    interesting_sections.append({
                        "page": default_page,
                        "text": display_text,
                        "category": "main",
                        "bbox": None,
                        "page_text": f"Page {default_page+1}"
                    })
            
            # Save results in structured format
            with open(file_name, "w", encoding="utf-8") as f:
                json.dump(cached_data, f, indent=2)
            
        except Exception as e:
            print(f"Error during interesting point extraction: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"Found {len(interesting_sections)} interesting sections")
    return interesting_sections

In [39]:
def highlight_interesting_points(pdf_path, interesting_points, output_path):
    """Add highlights to the interesting points in the PDF with complete text block highlighting."""
    doc = pymupdf.open(pdf_path)
    
    # Using cyan highlight color for main content
    highlight_color = (0, 1, 1)  # RGB for cyan
    fail_count = 0
    success_count = 0
    
    for point in interesting_points:
        page_num = point["page"]
        page = doc[page_num]
        text = point["text"]
        if not text:
            print(f"Empty text for page {page_num}")
            continue
            
        # Extract content without page annotations for better matching
        clean_text = re.sub(r'\[Page \d+\]\s*', '', text).strip()
        if not clean_text:
            continue
            
        # Try exact match first
        text_instances = page.search_for(clean_text)
        
        found_matches = []
        if not text_instances:
            # Method 1: Try normalized text (remove extra whitespace)
            normalized_text = ' '.join(clean_text.split())
            text_instances = page.search_for(normalized_text)
            
            # Method 2: Try with key phrases and collect all matches to combine later
            if len(normalized_text.split()) > 10:
                # Extract significant phrases (5-8 words)
                words = normalized_text.split()
                for i in range(len(words) - 5):
                    phrase = ' '.join(words[i:i+min(8, len(words)-i)])
                    if len(phrase) > 15:  # Only phrases with enough content
                        phrase_instances = page.search_for(phrase)
                        if phrase_instances:
                            found_matches.extend(phrase_instances)
            
            # Method 3: Use key sentences and collect all matches
            if '.' in normalized_text:
                sentences = [s.strip() for s in normalized_text.split('.') if len(s.strip()) > 15]
                for sentence in sentences:
                    sentence_instances = page.search_for(sentence)
                    if sentence_instances:
                        found_matches.extend(sentence_instances)
        
        # If we found partial matches, combine their bounding boxes to highlight full area
        if found_matches and not text_instances:
            if len(found_matches) >= 2:
                # Sort rectangles by y-coordinate and then by x-coordinate
                found_matches.sort(key=lambda r: (r[1], r[0]))  # Sort by top-y then left-x
                
                # Combine all matches into one larger rectangle
                x0 = min(rect[0] for rect in found_matches)
                y0 = min(rect[1] for rect in found_matches)
                x1 = max(rect[2] for rect in found_matches)
                y1 = max(rect[3] for rect in found_matches)
                
                # Create a single combined rectangle that covers all matches
                combined_rect = (x0, y0, x1, y1)
                text_instances = [combined_rect]
            else:
                text_instances = found_matches
        
        # Highlight found instances or use bbox as fallback
        if text_instances:
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.set_colors(stroke=highlight_color)
                highlight.update()
            success_count += 1
        elif point.get("bbox_list") and isinstance(point["bbox_list"], list) and point["bbox_list"]:
            # If we have a list of bounding boxes, highlight all of them
            for bbox in point["bbox_list"]:
                if bbox:  # Make sure bbox is not None
                    r = page.add_highlight_annot(bbox)
                    r.set_colors(stroke=highlight_color)
                    r.update()
            success_count += 1
        elif point.get("bbox"):
            # Single bbox case
            r = page.add_highlight_annot(point["bbox"])
            r.set_colors(stroke=highlight_color)    
            r.update()
            success_count += 1
        else:
            # Last resort: try to find a text chunk on the page containing some keywords
            if len(clean_text.split()) > 3:
                key_words = [w for w in clean_text.split() if len(w) > 5][:5]
                for word in key_words:
                    word_instances = page.search_for(word)
                    if word_instances:
                        for inst in word_instances:
                            highlight = page.add_highlight_annot(inst)
                            highlight.set_colors(stroke=(1, 0.5, 0))  # Orange for partial matches
                            highlight.update()
                        success_count += 1
                        break
            else:
                fail_count += 1
    
    print(f"Successfully highlighted {success_count} segments")
    print(f"Failed to highlight {fail_count} segments")
    
    # Save the highlighted PDF
    doc.save(output_path)
    doc.close()

In [40]:
def extract_text(pdf_path:str)-> tuple[List[dict], str]:
    """PyMuPDF-based function to extract text with bounding boxes from a PDF file."""
    try:
        doc = pymupdf.open(pdf_path, filetype="pdf")
        prev_block = None
        all_blocks = []
        for page_num, page in enumerate(doc):
            words = page.get_text("words")
            page_block = []
            # Take threshold based on page_width and page_height
            WIDTH_threshold = (0.02 if page.rect.width > page.rect.height else 0.0092625) * page.rect.width
            HEIGHT_threshold = (0.01 if page.rect.width > page.rect.height else 0.05) * page.rect.height
            for curr_word in words:
                # Each block is (x0, y0, x1, y1, text, block_no, block_type)
                x0, y0, x1, y1, text, block_no, line_no, block_type = curr_word
                text_height = y1 - y0
                if not text.strip():
                    continue
                is_mergable = False
                
                """
                Check if the current block is close to the previous block.
                The conditions are:
                (1. The x-coordinates of the current block are within WIDTH_threshold of the previous block.
                2. The y-coordinates of the current block are within HEIGHT_threshold of the previous block.
                (3. The current block is not completely to the left of the previous block.
                4. The y-coordinates of the current block are within 4 pixels of the previous block.
                5. The current block is not completely to the right of the previous block.
                """
                if prev_block and abs(text_height - prev_block[-1]) <= 0 and \
                    (\
                        (abs(x0 - prev_block[2]) <= WIDTH_threshold and (y0 -prev_block[3]) <= HEIGHT_threshold) \
                    ):
                    prev_block[2] = max(prev_block[2], x1) 
                    prev_block[3] = max(prev_block[3], y1) 
                    prev_block[4] += " " + text.strip()
                    is_mergable = True

                if is_mergable and page_block:
                    page_block.pop()
                    page_block.append(
                        {
                            "page": page_num,
                            "bbox": (prev_block[0], prev_block[1], prev_block[2], prev_block[3]),
                            "text": prev_block[4],
                            "block_no": block_no,
                            "block_type": block_type,
                            "text_height": prev_block[-1]
                        }
                    )    
                    prev_block = [prev_block[0], prev_block[1], prev_block[2], prev_block[3], prev_block[4], block_no, block_type, prev_block[-1]]
                else:
                    page_block.append({
                        "page": page_num,
                        "bbox": (x0, y0, x1, y1),
                        "text": text.strip(),
                        "block_no": block_no,
                        "block_type": block_type,
                        "text_height": text_height
                    })
                    # Update the previous block
                    prev_block = [x0, y0, x1, y1, text.strip(), block_no, block_type, text_height]
            # merged_blocks = process_text_blocks(page_block, eps=25, min_samples=1)
            all_blocks.extend(page_block)
        doc.close()
        # Remove empty blocks
        if not all_blocks:
            raise Exception("PyMuPDF Failed or No text found in the PDF.")
        return (all_blocks, 'pyMuPDF')
    except Exception as e:
        print("The Error is", e.with_traceback())
        # return extract_text_with_Mistral_OCR(pdf_path)



In [41]:
def classify_text_blocks(blocks, parsing_method: str):
    """Classify text blocks into main content, footnotes, and extra information."""
    if not blocks:
        print("No blocks to analyze")
        return None

    classified_blocks = []

    if parsing_method == 'Mistral OCR':
        # Implement classification for Mistral OCR if needed
        for block in blocks:
            text = block["text"].strip() if isinstance(block.get("text"), str) else ""
            
            # A placeholder for footnote detection (to be refined)
            is_footnote_marker = False  
            classification = "footnote" if is_footnote_marker else "main"

            block["category"] = classification
            classified_blocks.append(block)

    elif parsing_method == 'pyMuPDF':
        # First pass: Compute a ratio for each block to later determine the middle range
        ratios = []
        for block in blocks:
            text = block["text"].strip() if isinstance(block.get("text"), str) else ""
            bbox = block.get("bbox", [0, 0, 0, 0])
            text_height = block.get("text_height", 0)
            # Calculate area of the bounding box (space)
            space = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
            text_count = len(text.split())
            ratio = (text_count / space) if space > 0 else 0
            ratios.append(ratio)

        # Calculate the lower and upper quantiles for the middle 25% - adjust as needed.
        # Here, we use 15% and 40% quantiles from your earlier example.
        lower_q = pd.Series(ratios).quantile(0.25)
        upper_q = pd.Series(ratios).quantile(0.75)

        # Second pass: Classify each block using the ratio thresholds and footnote checks.
        for idx, block in enumerate(blocks):
            text = block["text"].strip() if isinstance(block.get("text"), str) else ""
            bbox = block.get("bbox", [0, 0, 0, 0])
            space = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
            text_count = len(text.split())
            ratio = (text_count / space) if space > 0 else 0

            # Improved footnote detection
            footnote_markers = ["*", "†", "‡", "§", "¶", "⁂", "⁎", "⁑", "⁕"]
            is_footnote_marker = (
                any(text.startswith(prefix) for prefix in footnote_markers) or
                any(text.startswith(str(i)) for i in range(1, 10))
            )

            # Classification logic:
            # - If it's a footnote marker, mark as "footnote".
            # - Otherwise, if the block falls within the middle thresholds and has sufficient words, it is "main".
            # - Else, it is "non-main"
            if is_footnote_marker:
                classification = "footnote"
            elif lower_q <= ratio <= upper_q and text_count > 20 and text_height > 0:
                classification = "main"
            else:
                classification = "extra"

            block["category"] = classification
            classified_blocks.append(block)

    return classified_blocks

In [42]:
# %%time
# # Example usage
# input_pdf_path = r"D:\DATA300\AudioBookSum\pdf\Gilman.pdf"
# doc = pymupdf.open(input_pdf_path)
# text = ""
# median_collection = []
# content = []
# i= 1
# possible_water_mark = None
# has_wm = False
# for i in range(len(doc[:3])):
#     extracted_pages = sorted(extract_text_with_ocr_approach(input_pdf_path, page_num =i, visual_proof= True), key = lambda x: [x["occupy_space"].y0, x["occupy_space"].x0])
#     if possible_water_mark and extracted_pages[-1]["content"] == possible_water_mark:
#         has_wm = True
#     if not extracted_pages:
#         continue
#     possible_water_mark = extracted_pages[-1]["content"]
# # print(has_wm)
# for i in range(1,len(doc)):
#     print(f"Processing page {i+1}/{len(doc)}...")
#     extracted_pages = sorted(extract_text_with_ocr_approach(input_pdf_path, page_num =i), key = lambda x: [x["occupy_space"].y0, x["occupy_space"].x0])
#     if has_wm:
#         extracted_pages.pop()
#     content.extend(extracted_pages)


In [44]:
base_name = os.path.splitext(input_pdf_path)[0]
output = f"{base_name}_highlighted.pdf"
chunked_data = chunk_text(content)
# Initialize LLM
try:
    llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.7)
except Exception as e:
    print(f"Error initializing Gemini LLM: {e}")
    print("Make sure you have set GOOGLE_API_KEY in your environment or .env file")
    exit(1)
words, extract_method = extract_text(input_pdf_path)
# print(extract_method)

print(f"Extracted {len(words)} text blocks")

# Classify text blocks
# classified_blocks = classify_text_blocks(words, extract_method)
# print("Classified text blocks")
# # Identify interesting points (main content only)
interesting_points = identify_interesting_points(chunked_data, llm, words, f"{base_name}_interesting_points.txt")  # Save to file
# load from memory
print(f"Identified {len(interesting_points)} interesting points in main content")

# # Highlight interesting points in the PDF
highlight_interesting_points(input_pdf_path, interesting_points, output)
print(f"Created highlighted PDF: {output}")

print("Processing complete!")

Extracted 263 text blocks
Loading cached interesting points from D:\DATA300\AudioBookSum\pdf\Clopath_interesting_points.txt
Generating new interesting points with LLM...
LLM response received
Extracted 7 segments from LLM response
Found page annotation in text: Page 1
Finding matching block for: - lf we study the word "art" and its etymology we  ...
Page hint: 0
Using default page 1 for unmatched segment: - [Page 1]
lf we study the word "art" and its etym...
Found page annotation in text: Page 2
Finding matching block for: - We find that art means skill, but in the fine ar ...
Page hint: 1
Using default page 2 for unmatched segment: - [Page 2]
We find that art means skill, but in th...
Found page annotation in text: Page 2
Finding matching block for: - The source of art is the perception of the beaut ...
Page hint: 1
Using default page 2 for unmatched segment: - [Page 2]
The source of art is the perception of ...
Found page annotation in text: Page 2
Finding matching block for: - We kn